# Práctica Algoritmo de Backtracking para resolver un sudoku

ALUMNO:

Marcos López Gómez
* marcos.lopez14@alu.uclm.es

En este práctica, he realizado un programa capaz de resolver un sudoku de 9x9, usando un algoritmo de búsqueda con retroceso.

A la hora de implementarlo, hay que realizar una serie de funciones para que este algoritmo funcione correctamente; las cuales serán expuestas en este documento.

# 0. Trabajo previo


## 0.1 Fichero 'sudoku.txt'

Este programa lee el sudoku de un fichero de texto, por lo que este es necario que exista, el nombre que le he dado es 'sudoku.txt'. Su contenido es el siguiente (o similar):

```
 63 8 4  
 9   4  7
4   2    
  72 3 58
 5  4    
  8   2 4
6     7  
  5 36   
 3 87  26
 ```

Donde los espacios representan los huecos en blanco iniciales en el sudoku.

## 0.2 Importaciones necesarias

In [78]:
import math as m

Para la realización de este algoritmo solo es necesaria la importación de la librería 'math'.

# 1. Representación del problema

Una parte vital para la resolución del problema es escoger la forma en la que será representado el problema, en este caso el sudoku; y para ello es necesario una función que sea capaz de leer el sudoku inicial del fichero de texto.

## 1.1 Leer el sudoku

In [79]:
def leer_sudoku(nombre_archivo="sudoku.txt"):
    sudoku = []
    with open(nombre_archivo, 'r') as archivo:
        for numero_linea, linea in enumerate(archivo, start=1):
            fila = []
            for caracter in linea:
                if caracter == ' ':
                    fila.append(0)  # Reemplazo el espacio con 0
                elif caracter.isdigit():  
                    fila.append(int(caracter))
            if len(fila) != 9:
                raise ValueError(f"Error en la línea {numero_linea}: debe tener 9 elementos, pero tiene {len(fila)}: {fila}")
            sudoku.append(fila)

    if len(sudoku) != 9:
        raise ValueError(f"El archivo debe tener 9 líneas, pero tiene {len(sudoku)}")

    return sudoku

Esta es la función que he implementado para leer es sudoku del fichero de texto, y además es la que deja el problema de la forma en la que será tratado, como una matriz; una lista de listas, donde cada una de las listas interiores representa una linea del sudoku, y como se indica en el código, los espacios en blanco son sustituidos por 0s (ya que estos no aparecen en los sudokus). Si el sudoku leido, no tiene exactamente 9 filas con 9 columnas cada una, dará error.

## 1.2 Visualización del sudoku

A continuación se deja un fragmento de código donde se puede apreciar la forma que tiene el sudoku una vez leido del fichero de texto.

In [80]:
sudoku=leer_sudoku()
for linea in sudoku:
    print(linea)

[0, 6, 3, 0, 8, 0, 4, 0, 0]
[0, 9, 0, 0, 0, 4, 0, 0, 7]
[4, 0, 0, 0, 2, 0, 0, 0, 0]
[0, 0, 7, 2, 0, 3, 0, 5, 8]
[0, 5, 0, 0, 4, 0, 0, 0, 0]
[0, 0, 8, 0, 0, 0, 2, 0, 4]
[6, 0, 0, 0, 0, 0, 7, 0, 0]
[0, 0, 5, 0, 3, 6, 0, 0, 0]
[0, 3, 0, 8, 7, 0, 0, 2, 6]


# 2. Funciones utilizadas

En este punto voy a comentar las funciones que se utilizan normalmente para los algoritmos de búsqueda con retroceso. En este caso las que he implementado y uso son 'es_prometedora' y 'es_factible'; así como un preproceso en el cual preparo un poco cada ejecución del algoritmo y algunas otras funciones.

## 2.1 Conversión de submatrices

Antes de entrar en materia, voy a comentar una función que he utilizado para simplificar la comprobación de las submatrices. Esta función lo que hace es devolver cada submatriz 3x3 del sudoku como una lista, lo cual me facilita mucho la tarea de comprobarlas.

In [81]:
def Obtener_submatrices(matriz):
    submatrices = []

    # Recorro los bloques de 3x3 dentro del sudoku
    for fila in range(0, 9, 3):
        for columna in range(0, 9, 3):

            submatriz = []
            for i in range(3):      # Recorro las filas dentro de la submatriz
                for j in range(3):  # Recorro las columnas dentro de la submatriz
                    submatriz.append(matriz[fila + i][columna + j])
            submatrices.append(submatriz)

    return submatrices

Dejo un ejemplo de ejecución para ver el resultado devuelto por la función. Se puede apreciar como cada matriz 3x3 del sudoku original ahora representa una lista.

In [82]:
submatrices=Obtener_submatrices(sudoku)
for bloque in submatrices:
    print(bloque)

[0, 6, 3, 0, 9, 0, 4, 0, 0]
[0, 8, 0, 0, 0, 4, 0, 2, 0]
[4, 0, 0, 0, 0, 7, 0, 0, 0]
[0, 0, 7, 0, 5, 0, 0, 0, 8]
[2, 0, 3, 0, 4, 0, 0, 0, 0]
[0, 5, 8, 0, 0, 0, 2, 0, 4]
[6, 0, 0, 0, 0, 5, 0, 3, 0]
[0, 0, 0, 0, 3, 6, 8, 7, 0]
[7, 0, 0, 0, 0, 0, 0, 2, 6]


## 2.2 Factibilidad

La siguiente es una función clave para la correcta ejecución del algoritmo de búsqueda con retroceso. En este caso, la función de factibilidad también hace de 'es_completa', ya que al comienzo de esta se comprueba el número de filas completadas, de tal manera que esta última no es necesaria.

In [83]:
def es_factible(sudoku,n):
    if (n<8):
        return False
    
    # No es necesario comprobar las filas, pues estas siempre son permutaciones

    # Compruebo si no hay repetidos en las columnas
    for c in range(9):
        temp = set()
        for r in range(9):
            valor = sudoku[r][c]
            if valor in temp:
                return False
            temp.add(valor)

    # Transformo las matrices 3x3 en vectores para facilitar su comprobacion
    sub_m = Obtener_submatrices(sudoku)
    for bloque in sub_m:
        temp = set()
        for valor in bloque:
            if valor in temp:
                return False
            temp.add(valor)

    
    return True

Una particularidad de esta función de factibilidad, es que no compruba si las lineas cumplen la condición del sudoku; esto es debido a que como se expone más adelante en el documento, estas siempre son permutaciones, por lo que siempre cumplen con la condición de que no haya repetidos, por lo que al saltarse la comprobación de las lineas se ahorra tiempo.

## 2.3 Comprobación de estados prometedores

Otra función clave en los algoritmos de backtracking es el 'es_prometedor', ya que permite descartar estados que no van a llegar a ningún lado de manera prematura. En este caso es muy similar a la función de factibilidad, pero con algunas particularidades para su correcto funcionamiento.

In [84]:
def es_prometedor(sudoku):

    # No es necesario comprobar las filas, pues estas son siempre permutaciones (pues son generadas así)

    # Compruebo si no hay repetidos en las columnas
    for c in range(9):
        temp = set()
        for r in range(9):
            valor = sudoku[r][c]
            if valor != 0 and valor in temp:
                return False
            temp.add(valor)

    # Transformo las matrices 3x3 en vectores para facilitar su comprobacion
    sub_m = Obtener_submatrices(sudoku)
    for linea in sub_m:
        temp = set()
        for valor in linea:
            if valor != 0 and valor in temp:
                return False
            temp.add(valor)

    return True

Al igual que la función de factibilidad, no compruba las lineas pues estas no pueden incumplir la condición del sudoku debido a la forma en la que son generadas. La particularidad de esta función con respecto de 'es_factible', es que no tiene en cuenta los 0s, para evitar que descarte estados no finales del sudoku, donde haya varios 0s por columna o submatriz. El resto de la función realmente es lo mismo que la enterior (sin comprobar la completitud como es lógico).

## 2.4 Preprocesamiento de las lineas del sudoku

La siguiente es una función que utilizo para simplificar el procesamiento de cada una de las lineas del sudoku en el algoritmo principal. Lo que hace es en función de como este la linea antes de procesarla, devuelve en un vector los números faltantes en esta, así como las posiciones libres.

In [85]:
def Generar_vectores(fila):
    vector_numeros=[]
    vector_posiciones=[]

    for i in range(0,9):
        if i+1 not in fila:
            vector_numeros.append(i+1)
        if fila[i]==0:
            vector_posiciones.append(i)

    return vector_numeros,vector_posiciones

A continuación dejo una ejecución sobre la primera linea del sudoku, para que se pueda apreciar el resultado de la ejecución.

In [86]:
linea1=sudoku[0]
print(linea1)
num,pos=Generar_vectores(linea1)
print('Números faltantes: ',num)
print('Posiciones libres: ',pos)

[0, 6, 3, 0, 8, 0, 4, 0, 0]
Números faltantes:  [1, 2, 5, 7, 9]
Posiciones libres:  [0, 3, 5, 7, 8]


## 2.5 Generación de permutaciones

Como he indicado previamente, la forma de generar las diferentes posibilidades de lineas, es mediante la generación de permutaciones con los numeros disponibles en las posiciones libres de cada linea. Para ello he utilizado el algoritmo de permutaciones lexicográficas, el cual he adaptado a mi programa. He escogido este método porque las formas en las que yo implementé algo similar me parecían muy poco eficientes y quise investigar alguna manera mejor de implementalo.

In [87]:
def siguiente_permutacion_lexicografica(v):
    i = len(v) - 2
    while i >= 0 and v[i] >= v[i + 1]:
        i -= 1
    if i == -1:
        return False
    j = len(v) - 1
    while v[j] <= v[i]:
        j -= 1
    v[i], v[j] = v[j], v[i]
    v[i+1:] = reversed(v[i+1:])
    return True

La forma en la que trabaja este algoritmo es a partir de un vector ordenado de menor a mayor, genera todas las permutaciones posibles con los elementos del vector. Lo que tiene de particular este método, es que es capaz de simplemente recibiendo la última permutación generada por el (o el vector ordenado en la primera ejecución) de generar la siguiente en orden lexicográfico (sin necesidad de ninguna variable para guardar el contexto). Por lo que partiendo simplemente del vector de números faltantes generado por la función anterior (el cual viene en orden) puedo generar todas las permutaciones para las posiciones libres. Cuando no haya más permutaciones, la función devuelve False para saber que se han agotado las opciones disponibles para la linea.

A continuación se muestra su funcionamiento mediante algunas ejecuciones.

In [88]:
num_cop=num[:]
print('Orden original: ',num)
siguiente_permutacion_lexicografica(num_cop)
print('Segunda permutación: ',num_cop)
siguiente_permutacion_lexicografica(num_cop)
print('Tercera permutación: ',num_cop)
siguiente_permutacion_lexicografica(num_cop)
print('Cuarta permutación: ',num_cop)


Orden original:  [1, 2, 5, 7, 9]
Segunda permutación:  [1, 2, 5, 9, 7]
Tercera permutación:  [1, 2, 7, 5, 9]
Cuarta permutación:  [1, 2, 7, 9, 5]


Tras esto, simplemente bastaría con adjudicar en cada iteración el primer elemenento del vector de números disponibles, a la primera posición disponible (almacenada en el vector de posiciones), el segundo elemenento a la segunda posición, etc.


# 3. Algoritmo de Búsqueda con Retroceso

Este es el algoritmo de backtracking propiamente dicho, el cual se encarga de mediante la utilización de llamadas recursivas, y de el resto de funciones especificadas, de encontrar la solución (si es que existe) del sudoku.

## 3.1 Algoritmo de resolución de sudokus

In [89]:
def resolver_sudoku(sudoku,n=0):

    if es_factible(sudoku,n):
        return sudoku
    
    fila_original=sudoku[n][:]
    valores,posiciones=Generar_vectores(sudoku[n])

    # El bucle como mucho se puede hacer (factorial del numero de elementos a añadir) veces
    for _ in range(m.factorial(len(valores))):
        fila=sudoku[n][:]
        for i in range(len(posiciones)):
            fila[posiciones[i]] = valores[i]
        sudoku[n]=fila[:]
        
        if es_prometedor(sudoku):
            resultado=resolver_sudoku(sudoku,n+1)
            if resultado!=False:
                return resultado
        sudoku[n]=fila_original[:]

    # Comprueba si hay alguna otra permutación (y si la hay la hace al ejecutar la funcion)
        if not siguiente_permutacion_lexicografica(valores):
            break

    return False

Este algoritmo tiene un esquema muy similar al los vistos en las diapositivas de teoría (particularmente al de la diapositiva 27), con algunas modificaciones, como por ejemplo que la función 'es_completa' ya es realizada de por si por 'es_factible', ya que me pareció más intuitivo hacerlo así.

La forma de trabajar de este algoritmo es: dentro de un bucle que se repite n! veces, siendo n el numero de elementos faltantes de la linea, se genera una posibilidad de linea, la cual se comprueba si es prometedora, si lo es se llama a la función para que haga lo mismo con la siguiente linea; si no es prometedora, o no es factible(se han agotado todas las combinaciones con esa linea), esa combinación se descarta, se genera otra permutación y se repite el proceso hasta que alguna combinación sea factible.

## 3.2 Ejemplo de ejecución y métricas

A continuación se deja un fragmento de código donde se ejecuta el algoritmo de búsqueda en un sudoku con solución y cuanto tarda el algoritmo en encontrarla.

In [92]:
import time

sudoku_copi=sudoku[:]

print('Sudoku inicial:')
for fila in sudoku:
    print(fila)

t=time.time()
solucion=resolver_sudoku(sudoku_copi)
t=time.time()-t

print('\nSudoku resuelto:')
for fila in solucion:
    print(fila)

print('\nSu resolución ha tardado: ',t,' segundos')


Sudoku inicial:
[0, 6, 3, 0, 8, 0, 4, 0, 0]
[0, 9, 0, 0, 0, 4, 0, 0, 7]
[4, 0, 0, 0, 2, 0, 0, 0, 0]
[0, 0, 7, 2, 0, 3, 0, 5, 8]
[0, 5, 0, 0, 4, 0, 0, 0, 0]
[0, 0, 8, 0, 0, 0, 2, 0, 4]
[6, 0, 0, 0, 0, 0, 7, 0, 0]
[0, 0, 5, 0, 3, 6, 0, 0, 0]
[0, 3, 0, 8, 7, 0, 0, 2, 6]

Sudoku resuelto:
[5, 6, 3, 7, 8, 1, 4, 9, 2]
[8, 9, 2, 3, 5, 4, 6, 1, 7]
[4, 7, 1, 6, 2, 9, 5, 8, 3]
[9, 4, 7, 2, 6, 3, 1, 5, 8]
[2, 5, 6, 1, 4, 8, 3, 7, 9]
[3, 1, 8, 5, 9, 7, 2, 6, 4]
[6, 8, 9, 4, 1, 2, 7, 3, 5]
[7, 2, 5, 9, 3, 6, 8, 4, 1]
[1, 3, 4, 8, 7, 5, 9, 2, 6]

Su resolución ha tardado:  3.2357614040374756  segundos
